<a href="https://colab.research.google.com/github/sjangnure/Natural-Language-Processing/blob/main/NLP_assignmnet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```

Name: Sanjana Jangnure
NetId: Sbj286
NLP Assignment 2: Bag of words, tfidf, Naive Bayes
```



##Import the dataset using pandas, make a column for 'RealNews?' and append both the files.

In [ ]:
import pandas as pd
import numpy as np
import re
df_real = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/True.csv')
df_real['RealNews?'] = True
df_fake = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Fake.csv')
df_fake['RealNews?'] = False
df = df_real.append(df_fake)

df=df.reset_index(drop=True)


df.head(2)

,title,text,subject,date,RealNews?
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True


##Preprocessing of data- combining, removing duplicates and splitting the data

In [ ]:
# Combining title and text column
df['document'] = df[['title', 'text']].agg(' '.join, axis=1)
df['document'] = df['document'].apply(lambda x: x.lower())

#drop duplicates
df=df.drop_duplicates()
df = df.sample(2000)
len(df)

2000

In [ ]:
# Split into train and test
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)
df_train.reset_index(drop=True)
df_test.reset_index(drop=True)
print(len(df_train))
print(len(df_test))
df_train.head(2)

1600
400


,title,text,subject,date,RealNews?,document
7327,Britain's Johnson says no need for gloom about...,BELGRADE (Reuters) - British Foreign Secretary...,politicsNews,"November 10, 2016",True,britain's johnson says no need for gloom about...
24336,WATCH: Bill Maher Returns And Hilariously Moc...,Bill Maher returned from break just in time fo...,News,"January 21, 2017",False,watch: bill maher returns and hilariously moc...


##Q1. Naive Bayes by hand

In [ ]:
# Tokenize the document

# Create a dictionary to store the bag of words
vocab=dict()

for row in df_train['document']:
  dictionary=re.split(r"\W+",row)
  for tokens in dictionary:
    if tokens not in vocab.keys():
      vocab[tokens]=1
    else:
      vocab[tokens]+=1


In [ ]:
# Calculate priors
# P(True)=No. of True docs in train/Total no. of docs in Train
true_lest=len(df_train[df_train['RealNews?']])
false_test=len(df_train[~df_train['RealNews?']])
print("No. of documents in Train is :",len(df_train))
print("No. of True documents in Train is: ",true_lest)
print("No. of False documents in Train is: ",false_test)
P_true=true_lest/len(df_train)
P_false=false_test/len(df_train)
print("P(True)=",P_true)
print("P(False)=",P_false)

No. of documents in Train is : 1600
No. of True documents in Train is:  771
No. of False documents in Train is:  829
P(True)= 0.481875
P(False)= 0.518125


Conditional probabilities

In [ ]:
dictionary=[]
vocab_true=dict()
for row in df_train[df_train['RealNews?']]['document']:
  dictionary=re.split(r"\W+",row)
  for tokens in dictionary:
    if tokens not in vocab_true.keys():
      vocab_true[tokens]=1
    else:
      vocab_true[tokens]+=1

# print(vocab_true)

In [ ]:
dictionary=[]
vocab_false=dict()
for row in df_train[~df_train['RealNews?']]['document']:
  dictionary=re.split(r"\W+",row)
  for tokens in dictionary:
    if tokens not in vocab_false.keys():
      vocab_false[tokens]=1
    else:
      vocab_false[tokens]+=1

# print(vocab_false)

In [ ]:
# Find |V|
# Calculate the count of unique tokens in the vocbulary 
V=len(vocab)
print("|V|=",V)

#count(c) = total tokens in the documents with class c
# count_true= count(c=True)=total tokens in the documents with class true
count_true=sum(vocab_true.values())
count_false=sum(vocab_false.values())
print(count_true,count_false)

|V|= 28742
303397 390435


In [ ]:
# Testing the data
dictionary=[]
predictions=[]
row=""
vocab2=dict()

for row in df_test['document']:
  # row=df_test['document'][i]
  # print(type(row))
  dictionary=re.split(r"\W+",row)
  value1=P_true
  value2=P_false
  for tokens in dictionary:
    if tokens in vocab_true:
      val1=(vocab_true[tokens]+1)/(count_true+V)
    else:
      val1=1/(count_true+V)
    if tokens in vocab_false:
      val2=(vocab_false[tokens]+1)/(count_false+V)
    else:
      val2=1/(count_false+V)
    vocab2[tokens]=(val1,val2)
    value1=value1*vocab2[tokens][0]
    value2=value2*vocab2[tokens][1]
  if value1>value2:
    predictions.append(True)
  else:
    predictions.append(False)

print(len(predictions))


400


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(df_test['RealNews?'], predictions,average='macro')


(0.7751988810210683, 0.5792069243156199, 0.5070562512422977, None)

##Q2. tf-idf by hand

In [ ]:
vocab_train=dict()
i = 0
for index, row in df_train.iterrows():
    for word in row['document']:
        if(word in vocab_train):
            vocab_train[word] += 1
        else:
            vocab_train[word] = 1
    
# Selecting tokens with frequency >=2 

BagOfWords = dict()
for tokens in vocab_train:
    if(vocab_train[tokens]>=2):
        BagOfWords[tokens] = vocab_train[tokens]
        
print('Bag of words before limiting min count to 2:',len(vocab_train))
print('Bag of Words length: ',len(BagOfWords))


Bag of words before limiting min count to 2: 80
Bag of Words length:  75


In [ ]:
import math
IDF = dict()


for index, row in df_train.iterrows():
    for tokens in set(row['document']):
        if tokens in BagOfWords:
            if tokens in IDF:
                IDF[tokens] += 1
            else:
                IDF[tokens] = 1
    

for tokens in IDF:
    IDF[tokens] = math.log(len(df_train)/IDF[tokens],10)


In [ ]:
BagOfWordsList = list(BagOfWords.keys())
def word_vectorizer(words):
    res = [0]*len(BagOfWords)
    for word in words:
        if(word in BagOfWordsList):
            wordIDF = IDF[word]
            wordTF = 1 + math.log(words.count(word),10)
            res[BagOfWordsList.index(word)] = wordTF * wordIDF
    return(res)

In [ ]:
# Vectorizing the train dataset
# print('Vectorizing Training set')
i = 0

trainData = []
trainLabels = []

for index, row in df_train.iterrows():
    trainData.append(word_vectorizer(row['document']))
    trainLabels.append(1 if row['RealNews?'] else 0)
    i += 1

# print("Training set Vectorized")

In [ ]:
# vectorizing test dataset

testData = []
testLabels = []

for index, row in df_test.iterrows():
    testData.append(word_vectorizer(row['document']))
    testLabels.append(1 if row['RealNews?'] else 0)
 

In [ ]:
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(random_state=0)
clf2.fit(trainData, trainLabels)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
predictions2 = clf2.predict(testData)

In [ ]:
precision_recall_fscore_support(df_test['RealNews?'],predictions2,average='macro')

(0.9441997798018216, 0.946658615136876, 0.9448331201885705, None)

##Q3 a) Naive Bayes using scikitlearn

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english',min_df=2)
X_train_cv = cv.fit_transform(df_train['document'])
X_test_cv = cv.transform(df_test['document'])

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv,df_train['RealNews?'])
predictions = naive_bayes.predict(X_test_cv)

precision_recall_fscore_support(df_test['RealNews?'],predictions,average='macro')

(0.9298820445609437, 0.9291465378421899, 0.9294905693636524, None)

##Q3 b) TF-IDF using scikitlearn

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

text_transformer = TfidfVectorizer(stop_words='english',ngram_range=(1,2),analyzer='word',lowercase=True,min_df=2)
X_train_text = text_transformer.fit_transform(df_train['document'])
X_test_text = text_transformer.transform(df_test['document'])

logit = LogisticRegression(C=5e1, solver='lbfgs', random_state=17, n_jobs=4)
logit.fit(X_train_text,df_train['RealNews?'])

test_preds = logit.predict(X_test_text)

precision_recall_fscore_support(df_test['RealNews?'],test_preds,average='macro')

(0.9641426282051282, 0.9663848631239935, 0.9648735447611401, None)



1.   **character ngram analyzer**: more the ngram more is the precision and recall. But the computation time increases as well. Did not change the scores much from (1,2) to (1,3) but a significant difference from (1,1) to (1,2)
2.   **stop words**: We can add manual list of stop words to make the model better. Removing or adding stop words parameter did not make much difference to the scores
3. **min_df**: value of 2 makes the computation faster.

